In [1]:
import torch
import torch.nn as nn


from torch.nn import functional as F
from torchvision import datasets
from torchvision import transforms
from torchvision.models.resnet import resnet50


from PIL import Image

In [2]:

pimg_size = (224,224) # resnet image size is 224,224 with 3 channels
img_size = (32,32) # size of Cifar 10 images. Also has 3 channels
mask_size = pimg_size #size of masking matrix
num_channels = 3 # num channels for resnet

batch_size = 20 

In [3]:
pad = int((pimg_size[0]-img_size[0])/2) # finding what padding I need on each border of the input image


transform = transforms.Compose([
    transforms.Pad(padding = pad),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
# adding padding, normalizing the data, and using lambda to up the num of channels

train_dataset = datasets.CIFAR10(root = 'data/', train=True, transform=transform, download=True) # download  dataset

test_dataset = datasets.CIFAR10(root = 'data/', train=False, transform=transform)


train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size, shuffle=True
)



test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=batch_size, shuffle=False
)

Files already downloaded and verified


In [6]:
device = torch.device('cuda:0') # using GPU
model = resnet50(weights=True).to(device) # downloading pretrained framework
model.eval() # puts the resnet model into evaluation mode which is good cause we don't want to change the model as this is an adversarial attack
for param in model.parameters(): #making sure that no gradients are tracked while we use the model
    param.requires_grad = False

In [7]:
program = torch.rand(num_channels, *pimg_size, device=device, requires_grad = True) #creating our adversarial program starting point by making a random tensor, which we will adjust later


pad = int((mask_size[0]-img_size[0])/2) 

mask = torch.zeros(num_channels, *img_size, device=device) # creating the mask
mask = F.pad(mask, (pad,pad,pad,pad), value=1)

optimizer = torch.optim.Adam([program], lr=0.05) #the optimizer which optimizes our adversarial program
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2)  # used to adjust the learning rate as time goes on

loss_criterion = nn.CrossEntropyLoss() # loss criterion


num_classes = 10 # num classes for MNSIST

In [ ]:
#Training loop
num_epochs = 20
steps_per_epoch = len(train_loader)

for epoch in range(num_epochs):

    if epoch == 0:
      print('Training has started')

    
    total_loss = 0.0

    for i, data in enumerate(train_loader):
      images = data[0].to(device) #grabbing images from the data
      labels = data[1].to(device) #grabbing labels from the data

      images = images + torch.tanh(program*mask) #additive contribution to the image

      outputs = model(images) # see what resnet classifies
      outputs = outputs[:,:num_classes] #only care about first 10 cause MNIST only has 10 classes.
     
      loss = loss_criterion(outputs, labels) # find loss
      loss.backward() # backwards pass
      optimizer.step() #update weights
      optimizer.zero_grad()


      total_loss += loss.item() #calculate total loss
      
      if(i+1)%(steps_per_epoch/2) == 0:
        print(f'Epoch {epoch+1}/{num_epochs},  Step {i+1}/{steps_per_epoch},  Loss:  {loss.item()},  Avg Loss for Epoch:  {total_loss/(i+1)}')
    lr_scheduler.step() # adjusting learning rate

Training has started
Epoch 1/20,  Step 1250/2500,  Loss:  1.5758728981018066,  Avg Loss for Epoch:  1.7716222479820252
Epoch 1/20,  Step 2500/2500,  Loss:  1.5580806732177734,  Avg Loss for Epoch:  1.6559666161060334
Epoch 2/20,  Step 1250/2500,  Loss:  1.3510596752166748,  Avg Loss for Epoch:  1.45336142244339
Epoch 2/20,  Step 2500/2500,  Loss:  1.5012667179107666,  Avg Loss for Epoch:  1.4316902688026427
Epoch 3/20,  Step 1250/2500,  Loss:  1.39601731300354,  Avg Loss for Epoch:  1.361149688768387
Epoch 3/20,  Step 2500/2500,  Loss:  1.3686997890472412,  Avg Loss for Epoch:  1.361206523990631


In [ ]:
# testing
with torch.no_grad(): 

  total_loss = 0.0
  total_steps = len(test_loader)
  total = 0
  total_correct = 0


  for data in test_loader:
      images = data[0].to(device)
      labels = data[1].to(device) 

      images = images + torch.tanh(program*mask)
      outputs = model(images)
      outputs = outputs[:,:num_classes]

      preds = torch.argmax(torch.softmax(outputs, dim =1), dim = 1) #grabs index of highest pred

      total_correct += torch.sum(preds==labels).item() 
      total+= labels.shape[0]




In [ ]:
acc = (total_correct/total) * 100.0
print(f'Accuracy: {acc:.4f}')

In [ ]:
torch.save({'program':program, 'mask':mask}, 'resnet50_adversarial_CIFAR10.pth')

In [ ]:
state = torch.load('resnet50_adversarial_CIFAR10.pth')
program = state['program']
mask = state['mask']

In [ ]:
transforms.ToPILImage()(program.detach().cpu())
x,y = train_dataset[0]


torch.max(x)

x.shape



def imshow(img):
    return transforms.ToPILImage()(img.detach().cpu())

In [ ]:
imshow(program)